# FEniCSx - Lineare Elastizität

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
## Inhaltsverzeichnis<a id='toc0_'></a>    
- 1. [Voraussetzungen](#toc1_)    
- 2. [Start des Linux Containers in VSCode](#toc2_)    
- 3. [Beispiel des FEniCSx Workflows](#toc3_)    
  - 3.1. [Aufgabe](#toc3_1_)    
  - 3.2. [Lösung](#toc3_2_)    
    - 3.2.1. [Notwendige Bibliotheken](#toc3_2_1_)    
    - 3.2.2. [Parameter](#toc3_2_2_)    
    - 3.2.3. [Geometrie und Vernetzung](#toc3_2_3_)    
    - 3.2.4. [Randbedingungen](#toc3_2_4_)    
    - 3.2.5. [Zusammenhänge der linearen Elastizität](#toc3_2_5_)    
    - 3.2.6. [Solver](#toc3_2_6_)    
    - 3.2.7. [Visualisierung](#toc3_2_7_)    
- 4. [Referenzen](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
## 1. <a id='toc1_'></a>[Voraussetzungen](#toc0_)

- Installation von **Docker** & **VSCode**
- Erweiterungen in **VSCode**:
    - Remote Development

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
## 2. <a id='toc2_'></a>[Start des Linux Containers in VSCode](#toc0_)

| Windows          | Mac                 |
|------------------|---------------------|
| Strg + Shift + P | Command + Shift + P |

1. Eingabe: `Dev Containers: Clone Repository in Container Volume`
2. Eingabe: `psatke/FEniCSx-Env`

Nachdem der Container läuft kann das Notebook unter `src/FEniCSx-LineareElastizität.ipynb` aufgerufen werden.

Info: Um Dateien aus dem Container heraus oder in ihn hinein zu kopieren kann folgender Befehl verwendet werden:

`docker cp [OPTIONS] CONTAINER:SRC_PATH DEST_PATH|-`

`docker cp [OPTIONS] SRC_PATH|- CONTAINER:DEST_PATH`

Beispiel: 

` docker cp ./some_file CONTAINER:/work`

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>

## 3. <a id='toc3_'></a>[Beispiel des FEniCSx Workflows](#toc0_)

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
### 3.1. <a id='toc3_1_'></a>[Aufgabe](#toc0_)

[<img src="Aufgabe_3-Punkt.svg" width="600"/>](Aufgabe_3-Punkt.svg)

Berechne die Verschiebungen des Tragwerks mittels FEM und stelle sie in einer geeigneten Form dar.

Materialparameter:
- $E = 2e11\,\text{N}\,\text{m}^\text{-2}$
- $\nu = 0.3$
- $\rho = 7850\,\text{kg}\,\text{m}^\text{-3}$

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
### 3.2. <a id='toc3_2_'></a>[Lösung](#toc0_)

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.1. <a id='toc3_2_1_'></a>[Notwendige Bibliotheken](#toc0_)

In [28]:
# FEniCSx Bibliotheken
import dolfinx
import ufl

# Datenverwaltung für die CPU oder GPU Nutzung
from petsc4py import PETSc
from mpi4py import MPI

# Visualisierung
import pyvista

import numpy as np

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.2. <a id='toc3_2_2_'></a>[Parameter](#toc0_)

In [29]:
L = 0.5         # [m]
W = 0.1         # [m] 
nu = 0.3
EMod = 2e11     # [N/m2]
rho = 7850      # [kg/m3]

q = -200000     # [N/m2]

# Lamé-Konstanten für isotrope Festkörper
lambda_ = nu/(1-2*nu)*1/(1+nu)*EMod
mu = 1/2*1/(1+nu)*EMod

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.3. <a id='toc3_2_3_'></a>[Geometrie und Vernetzung](#toc0_)

In [30]:
# Communicator zwischen CPU oder GPU
comm = MPI.COMM_WORLD
print("Dieses Notebook kann auf {} Prozessor-Kern zugreifen".format(comm.Get_size()))

Dieses Notebook kann auf 1 Prozessor-Kern zugreifen


In [31]:
mesh = dolfinx.mesh.create_box(MPI.COMM_WORLD, [np.array([0,0,0]), np.array([L,W,W])], [1,1,2], cell_type=dolfinx.mesh.CellType.hexahedron)
print("Das erstellte Netz hat {} Dimensionen und die erzeugten Elemente und Knoten sind wie folgt Nummeriert: \n{}".format(mesh.geometry.dim, mesh.geometry.dofmap))

Das erstellte Netz hat 3 Dimensionen und die erzeugten Elemente und Knoten sind wie folgt Nummeriert: 
<AdjacencyList> with 2 nodes
  0: [0 1 2 3 4 5 6 7 ]
  1: [4 5 6 7 8 9 10 11 ]



In [32]:
V = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 1))
topology, cell_types, geometry = dolfinx.plot.create_vtk_mesh(V)
topology, cell_types, geometry

(array([ 8,  0,  1,  3,  2,  4,  5,  7,  6,  8,  4,  5,  7,  6,  8,  9, 11,
        10], dtype=int32),
 array([12, 12]),
 array([[ 1.38777878e-17, -2.77555756e-18,  6.93889390e-19],
        [ 5.00000000e-01,  1.38777878e-17,  6.24500451e-18],
        [ 5.55111512e-17,  1.00000000e-01,  6.24500451e-18],
        [ 5.00000000e-01,  1.00000000e-01, -9.71445147e-18],
        [ 1.38777878e-17, -2.77555756e-18,  5.00000000e-02],
        [ 5.00000000e-01,  1.38777878e-17,  5.00000000e-02],
        [ 5.55111512e-17,  1.00000000e-01,  5.00000000e-02],
        [ 5.00000000e-01,  1.00000000e-01,  5.00000000e-02],
        [ 4.85722573e-17,  6.93889390e-18,  1.00000000e-01],
        [ 5.00000000e-01, -1.52655666e-17,  1.00000000e-01],
        [-9.71445147e-17,  1.00000000e-01,  1.00000000e-01],
        [ 5.00000000e-01,  1.00000000e-01,  1.00000000e-01]]))

In [33]:
#     +---------------------+         +                     +          ---------------------
#    /                     /|                                        /                     /|
#   /                     / |                                       /                     / |
#  /                     /  |                                      /                     /  |
# +---------------------+   +     +                     +   +      --------------------- 
# |                     |  /                                      |                     |  /
# |        Mesh         | /               Geometry                |       Topology      | /
# |                     |/                                        |                     |/
# +---------------------+         +                     +          ---------------------

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.4. <a id='toc3_2_4_'></a>[Randbedingungen](#toc0_)

In [34]:
def floatingBearingGetLoc(x):
    xBoolean = np.isclose(x[0], 0)
    zBoolean = np.isclose(x[2], 0)
    return xBoolean & zBoolean

floatingBearingLoc = dolfinx.mesh.locate_entities_boundary(mesh, 1, floatingBearingGetLoc)
floatingBearingDof = dolfinx.fem.locate_dofs_topological(V.sub(2), 1, floatingBearingLoc)

floatingBearing = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), floatingBearingDof, V.sub(2))

In [35]:
def symmetryXGetLoc(x):
    return np.isclose(x[0], L)

symmetryXLoc = dolfinx.mesh.locate_entities_boundary(mesh, 2, symmetryXGetLoc)
symmetryXDof = dolfinx.fem.locate_dofs_topological(V.sub(0), 2, symmetryXLoc)

symmetryX = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), symmetryXDof, V.sub(0))

In [36]:
def symmetryYGetLoc(x):
    return np.isclose(x[1], 0)

symmetryYLoc = dolfinx.mesh.locate_entities_boundary(mesh, 2, symmetryYGetLoc)
symmetryYDof = dolfinx.fem.locate_dofs_topological(V.sub(1), 2, symmetryYLoc)

symmetryY = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), symmetryYDof, V.sub(1))

In [37]:
def loadGetLoc(x):
    # xBoolean = np.isclose(x[0], L)
    return np.isclose(x[2], W)
    # return xBoolean & zBoolean

loadLoc = dolfinx.mesh.locate_entities_boundary(mesh, 2, loadGetLoc)
mt = dolfinx.mesh.meshtags(mesh, 2, loadLoc, 1)
ds = ufl.Measure("ds", domain=mesh, subdomain_data=mt, subdomain_id=1)
# dx = ufl.Measure("dx", domain=domain)

In [38]:
T = dolfinx.fem.Constant(mesh, PETSc.ScalarType((0,0,q)))
# T = dolfinx.Constant(domain, ufl.as_vector((0,0,-1000)))

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.5. <a id='toc3_2_5_'></a>[Zusammenhänge der linearen Elastizität](#toc0_)

In [39]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2*mu*epsilon(u)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# f = dolfinx.fem.Constant(domain, PETSc.ScalarType((0,0,-9.81*rho)))
f = dolfinx.fem.Constant(mesh, PETSc.ScalarType((0,0,0)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds
# L = f* v * dx + T* v * ds

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.6. <a id='toc3_2_6_'></a>[Solver](#toc0_)

In [40]:
problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[floatingBearing, symmetryX, symmetryY], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
#### 3.2.7. <a id='toc3_2_7_'></a>[Visualisierung](#toc0_)

In [43]:
import pyvista
# pyvista.set_jupyter_backend('ipygany')  
pyvista.start_xvfb()
pyvista.global_theme.font.color = 'black'

# Create plotter and pyvista grid
p = pyvista.Plotter()
p.background_color = "white"

topology, cell_types, geometry = dolfinx.plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
edge = pyvista.UnstructuredGrid(np.array([8, 0, 1, 3, 2, 4, 5, 7, 6], dtype=np.int32), 
                                np.array([12]), 
                                np.array([[ 6.93889390e-18,  1.38777878e-18,  4.16333634e-18],
                                    [ 5.00000000e-01,  9.71445147e-18,  6.93889390e-18],
                                    [ 4.85722573e-17,  1.00000000e-01,  6.93889390e-18],
                                    [ 5.00000000e-01,  1.00000000e-01, -1.80411242e-17],
                                    [ 4.85722573e-17,  6.93889390e-18,  1.00000000e-01],
                                    [ 5.00000000e-01, -1.80411242e-17,  1.00000000e-01],
                                    [-1.04083409e-16,  1.00000000e-01,  1.00000000e-01],
                                    [ 5.00000000e-01,  1.00000000e-01,  1.00000000e-01]]))

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(edge, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=160)
actor_1 = p.add_mesh(warped, show_edges=False)
_ = p.add_axes(interactive=True, color='black')
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   figure_as_array = p.screenshot("deflection.png")

AppLayout(children=(VBox(children=(HTML(value='<h3>u</h3>'), Dropdown(description='Colormap:', options=(('BrBG…

<style>
a:link {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: none;
}
a:hover {
  color: rgba(200, 200, 200, 1);
  background-color: transparent;
  text-decoration: underline;
}
</style>
## 4. <a id='toc4_'></a>[Referenzen](#toc0_)

Alternative Beispiele, sowie Hintergrundinformationen zur Mathematik und der FEniCSx Implementierung
https://jsdokken.com/dolfinx-tutorial/fem.html